In [1]:
import pandas as pd
import os

# Folder containing all the stock data CSVs
data_folder = 'C:\\Users\\Amenzz\\Downloads\\Compressed\\data1\\yfinance_data'

# List CSV files in the folder
files = [f for f in os.listdir(data_folder) if f.endswith('.csv')]

# Dictionary to hold stock DataFrames
stock_data = {}

for file in files:
    stock_name = file.split('_')[0]  # e.g., AAPL
    df = pd.read_csv(os.path.join(data_folder, file))

    # Convert Date column to datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Ensure all required columns exist
    required_cols = {'Open', 'High', 'Low', 'Close', 'Volume'}
    if required_cols.issubset(df.columns):
        df = df.sort_values('Date')
        stock_data[stock_name] = df
    else:
        print(f"⚠️ Skipping {file} — missing required columns.")

# ✅ Check what's loaded
for symbol, df in stock_data.items():
    print(f"\n📈 {symbol} Data:")
    print(df.head())


📈 AAPL Data:
        Date      Open      High       Low     Close  Adj Close     Volume  \
0 1980-12-12  0.128348  0.128906  0.128348  0.128348   0.098943  469033600   
1 1980-12-15  0.122210  0.122210  0.121652  0.121652   0.093781  175884800   
2 1980-12-16  0.113281  0.113281  0.112723  0.112723   0.086898  105728000   
3 1980-12-17  0.115513  0.116071  0.115513  0.115513   0.089049   86441600   
4 1980-12-18  0.118862  0.119420  0.118862  0.118862   0.091630   73449600   

   Dividends  Stock Splits  
0        0.0           0.0  
1        0.0           0.0  
2        0.0           0.0  
3        0.0           0.0  
4        0.0           0.0  

📈 AMZN Data:
        Date      Open      High       Low     Close  Adj Close      Volume  \
0 1997-05-15  0.121875  0.125000  0.096354  0.097917   0.097917  1443120000   
1 1997-05-16  0.098438  0.098958  0.085417  0.086458   0.086458   294000000   
2 1997-05-19  0.088021  0.088542  0.081250  0.085417   0.085417   122136000   
3 1997-05-20 

In [ ]:
import pandas as pd
import ta
# data_folder = 'C:\\Users\\Amenzz\\Downloads\\Compressed\\data1\\yfinance_data'
# df = pd.read_csv(data_folder)
df = stock_data['AAPL'].copy()
df = df.sort_values('Date')

# Add RSI
df['RSI'] = ta.momentum.RSIIndicator(close=df['Close'], window=14).rsi()

# Add EMA
df['EMA_20'] = ta.trend.EMAIndicator(close=df['Close'], window=20).ema_indicator()

# Add MACD
macd = ta.trend.MACD(close=df['Close'])
df['MACD'] = macd.macd()
df['MACD_signal'] = macd.macd_signal()

print(df[['Date', 'Close', 'RSI', 'EMA_20', 'MACD', 'MACD_signal']].tail())


In [ ]:
import pandas as pd
import talib

# Example: using AAPL from your loaded data
df = stock_data['AAPL'].copy()

# Ensure it's sorted by date
df = df.sort_values('Date')

# Calculate Moving Averages
df['SMA_20'] = talib.SMA(df['Close'], timeperiod=20)
df['EMA_20'] = talib.EMA(df['Close'], timeperiod=20)

# Calculate RSI (Relative Strength Index)
df['RSI_14'] = talib.RSI(df['Close'], timeperiod=14)

# Calculate MACD
df['MACD'], df['MACD_signal'], df['MACD_hist'] = talib.MACD(
    df['Close'], fastperiod=12, slowperiod=26, signalperiod=9
)

# Show a few rows
print(df[['Date', 'Close', 'SMA_20', 'EMA_20', 'RSI_14', 'MACD']].tail())


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(14, 6))
plt.plot(df['Date'], df['Close'], label='Close Price', color='black')
plt.plot(df['Date'], df['SMA_20'], label='SMA 20', color='blue')
plt.plot(df['Date'], df['EMA_20'], label='EMA 20', color='green')
plt.title("AAPL Closing Price with Moving Averages")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
import pynance as pn

# Fetch historical data (you can also load your CSVs if needed)
aapl = pn.data.get('AAPL', start='2023-01-01', end='2024-12-31')

# Calculate common indicators
aapl['SMA_20'] = pn.indicators.sma(aapl['close'], 20)
aapl['EMA_20'] = pn.indicators.ema(aapl['close'], 20)
aapl['RSI_14'] = pn.indicators.rsi(aapl['close'], 14)

# Optional: MACD
macd_line, macd_signal = pn.indicators.macd(aapl['close'])
aapl['MACD'] = macd_line
aapl['MACD_signal'] = macd_signal


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# 🟢 Price with SMA and EMA
axs[0].plot(aapl.index, aapl['close'], label='Close Price', color='black')
axs[0].plot(aapl.index, aapl['SMA_20'], label='SMA 20', color='blue')
axs[0].plot(aapl.index, aapl['EMA_20'], label='EMA 20', color='green')
axs[0].set_title('AAPL Price with SMA and EMA')
axs[0].legend()
axs[0].grid(True)

# 🟣 RSI
axs[1].plot(aapl.index, aapl['RSI_14'], label='RSI 14', color='purple')
axs[1].axhline(70, color='red', linestyle='--', label='Overbought')
axs[1].axhline(30, color='green', linestyle='--', label='Oversold')
axs[1].set_title('RSI Indicator')
axs[1].legend()
axs[1].grid(True)

# 🔵 MACD
axs[2].plot(aapl.index, aapl['MACD'], label='MACD', color='blue')
axs[2].plot(aapl.index, aapl['MACD_signal'], label='Signal Line', color='orange')
axs[2].set_title('MACD Indicator')
axs[2].legend()
axs[2].grid(True)

plt.tight_layout()
plt.show()
